In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.shape

In [ ]:
# Y = df['Survived'].values

# train = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy().fillna(0)
# sdico = {'male' : 0, 'female': 1}

# train['Sex'] = train['Sex'].replace(sdico).copy()

# edico = {'S':0, 'C':1, 'Q':2}
# def app(x):
#     try:
#         return edico[x]
#     except:
#         return 3
# train['Embarked'] = train['Embarked'].apply(app).copy()

# X = train.values

In [ ]:
df.columns

In [ ]:
Y = df['target'].values
X = df[['var_'+str(i) for i in range(200)]].values

In [ ]:
# y = tf.keras.utils.to_categorical(Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from tabnet_layer import *
import tensorflow as tf

In [ ]:
tabnet = TabNet(
#            columns = None,
           num_features = 200,
           feature_dim = 512,
           output_dim = 256,
           num_decision_steps = 3,
           relaxation_factor = 1.5,
           batch_momentum = 0.7,
           virtual_batch_size = None,
#            num_classes = 0,
           epsilon=0.00001)

In [ ]:
inputs = tf.keras.Input(shape = (200,))

encoded, entropy = tabnet(inputs)
# encoded = tf.keras.layers.Dense(128,activation = 'relu')(inputs)
# encoded = tf.keras.layers.Dense(128,activation = 'relu')(encoded)
# encoded = tf.keras.layers.Dense(128,activation = 'relu')(encoded)
pred = tf.keras.layers.Dense(1, activation = 'sigmoid')(encoded)
model = tf.keras.Model(inputs, pred)

In [ ]:
model.summary()

In [ ]:
batch_size = 256

In [ ]:
temperature = 0.1
def loss_auc(true, pred, batch_size = batch_size):
        
    pred = tf.repeat(pred, batch_size, axis = -1)
    
    diff1 = tf.math.exp(- temperature * (pred - tf.transpose(pred)))
    true = tf.repeat(true, batch_size, axis = -1)
    
    zero_un_comp = tf.cast(tf.math.maximum(true - tf.transpose(true), 0), dtype = diff1.dtype)
    diff1 *= zero_un_comp
    return tf.math.reduce_sum(diff1)/tf.math.reduce_sum(zero_un_comp)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, reduction='none')

acc_object = tf.keras.metrics.SparseCategoricalAccuracy()

def acc(true, pred):
    return tf.reduce_mean(acc_object(true, pred))

def loss(true, pred):
    return tf.reduce_mean(loss_object(true, pred))

model.compile(
        loss = 'categorical_crossentropy',
        optimizer = Adam(0.02),
        metrics = ['accuracy', 'AUC'])

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score


class AUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation=None, logs = {}, dico_params = {}, from_path = None):
        super(AUCCallback, self).__init__()
        self.validation = validation
        self.epoch = []

    def on_epoch_end(self, epoch, logs={}):
        ## Roc auc calculation on test set
        x_val, y_val = self.validation[0], self.validation[1]
        pred = model.predict(x_val, verbose = 0)
        
        pred = pred[:,1]
        true = y_val[:,1]
        
        roc_auc = roc_auc_score(y_val, pred)
        
        logs['roc_auc'] = roc_auc
        print(logs)

In [ ]:
epochs = 100
batch_size = 6400



from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau


early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=9, verbose=1, 
                                                mode='auto', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1, 
                           mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)



st = X_train.shape[0] // batch_size * batch_size
sv = X_test.shape[0] // batch_size * batch_size

callbacks =[early, reduce, AUCCallback(validation = (X_test[:sv], y_test[:sv]))]

model.fit(X_train[:st], y_train[:st],  validation_data = (X_test[:sv], y_test[:sv]), batch_size = batch_size, epochs = epochs, callbacks = callbacks)

In [ ]:
pred = model.predict(X_test, verbose = 1)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
print(accuracy_score(np.argmax(y_test, axis = -1), np.argmax(pred, axis = -1)))
print(roc_auc_score(np.argmax(y_test, axis = -1), pred[:,1]))

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth = -1, n_estimators = 500, n_jobs = 12, silent = False)
clf.fit(X_train, y_train[:,1], eval_set =(X_test, y_test[:,1]), eval_metric = 'auc')

## Adult Dataset

In [ ]:
df = pd.read_csv('adult.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
def categorize(df, field):
    x = df[field].unique()
    dico = {}
    for i, elt in enumerate(x):
        dico[elt] = i
    
    def apply_dico(a):
        try:
            return dico[a]
        except:
            return len(x) + 1
        
    new_cols = df[field].apply(apply_dico)
    
#     if field !=  '<=50K':
#         new_cols = tf.keras.utils.to_categorical(new_cols)

#         df = df.drop(columns = field)

#         for i in range(new_cols.shape[1]):
#             df[field+'_'+str(i)] = new_cols[:,i]
#     else:
    df[field] = new_cols
    
    return df


cat_cols = ['workclass', 'education', 'education.num', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'native.country', '<=50K']

num_cols = [elt for elt in df.columns if not(elt in cat_cols)]
for field in cat_cols:
    df = categorize(df, field)
    
cats_ids = [i for i in range(len(df.columns[:-1])) if df.columns[i] in cat_cols]

In [ ]:
cats_ids

In [ ]:
y = df['<=50K']
df= df.drop(columns = ['<=50K'])

In [ ]:
# y = tf.keras.utils.to_categorical(y)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [ ]:
X_train = [X_train[elt].values.reshape(-1,1) for elt in cat_cols[:-1]] + [X_train[num_cols].values]
X_test = [X_test[elt].values.reshape(-1,1) for elt in cat_cols[:-1]] + [X_test[num_cols].values]

In [ ]:
len(X_train)

In [ ]:
out_size = [9,10,10,6,12,6,5,3,25]
np.sum(out_size) + 4

In [ ]:
from tabnet2 import *
import tensorflow as tf
tabnet = TabNet(
        num_features = 90,
        feature_dim = 64,
        output_dim = 64,
        feature_columns = None,
        n_step = 5,
        n_total = 5,
        n_shared = 2,
        relaxation_factor = 1.5,
        bn_epsilon = 1e-5,
        bn_momentum = 0.7,
        bn_virtual_divider = 10,
    )

inputs = [tf.keras.Input((1,)) for i in range(9)] + [tf.keras.Input((4,))]

in_size = [9, 16, 16,7,15,6,5,3,42]
out_size = [9,10,10,6,12,6,5,3,25]

embed = [tf.squeeze(tf.keras.layers.Embedding(in_size[i], out_size[i])(inputs[i]), axis = 1) for i in range(len(in_size))] + [inputs[-1]]

embed = tf.keras.layers.Concatenate(axis = -1)(embed)

encoded, masks = tabnet(embed, training = True)

pred = tf.keras.layers.Dense(1, activation = 'sigmoid')(encoded)
model = tf.keras.Model(inputs, pred)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD
model.compile(
        loss = 'binary_crossentropy',
        optimizer = Adam(0.1),
        metrics = ['accuracy', 'AUC'])

In [ ]:
X_train[0].shape

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=9, verbose=1, 
                                                mode='auto', restore_best_weights=True)

reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1, 
                           mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

batch_size = 2600
epochs = 100

st = X_train[0].shape[0] // batch_size * batch_size
sv = X_test[0].shape[0] // batch_size * batch_size

callbacks =[early, reduce]

model.fit([elt[:st] for elt in X_train], y_train[:st],  validation_data = ([elt[:sv] for elt in X_test[:sv]], y_test[:sv]), batch_size = batch_size, epochs = epochs, callbacks = callbacks)

In [ ]:
a = model.predict(X_test, verbose = 1, batch_size = 1024)

In [ ]:
accuracy_score(y_test[:,1], np.argmax(a, axis = 1))

In [ ]:
roc_auc_score(y_test[:,1], a[:,1])

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(max_depth = -1, n_estimators = 10, n_jobs = 12, silent = False)
clf.fit(X_train, y_train, eval_set =(X_test, y_test), eval_metric = 'auc')

In [ ]:
pred = clf.predict(X_train)
accuracy_score(y_train, pred)